In [ ]:
# Import required modules
from chronoeeg.io import EEGDataLoader
from chronoeeg.preprocessing import EpochExtractor
from chronoeeg.quality import QualityAssessor
from chronoeeg.features import ClassicalFeatureExtractor, FMMFeatureExtractor

## Basic Usage Example from README

In [2]:
# Initialize loader
loader = EEGDataLoader(data_folder="./data")

# Discover available patients
patients = loader.find_patients()
print(f"Found {len(patients)} patients")

# List available recordings for a patient (useful for large datasets)
recordings = loader.list_recordings("0284")
print(f"Patient 0284 has {len(recordings)} recordings")

Found 1 patients
Patient 0284 has 2 recordings


In [3]:
# Load specific recording(s) only (recommended for large datasets)
eeg_data, metadata = loader.load_patient(
    patient_id="0284",
    recording_names=["0284_001_004_EEG"]  # Load only specific recordings
)

print(f"\nData shape: {eeg_data.shape}")
print(f"Channels: {list(eeg_data.columns)}")
print(f"Metadata keys: {list(metadata.keys())}")


Data shape: (1578500, 19)
Channels: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
Metadata keys: ['patient_id', 'Patient', 'Hospital', 'Age', 'Sex', 'ROSC', 'OHCA', 'Shockable Rhythm', 'TTM', 'Outcome', 'CPC', 'sampling_frequency', 'num_samples', 'channels', 'record_name', 'Start time', 'End time', 'utility_frequency', 'num_recordings', 'loaded_recordings']


## Epoch Extraction and Feature Analysis

In [4]:
# Extract 5-minute epochs
epoch_extractor = EpochExtractor(epoch_duration=10, sampling_rate=128)
epochs = epoch_extractor.extract(eeg_data, metadata)

print(f"Extracted {len(epochs)} epochs")
print(f"First epoch shape: {epochs[0]['data'].shape}")

Extracted 41 epochs
First epoch shape: (38401, 19)


In [6]:
# Assess signal quality
quality_assessor = QualityAssessor()
quality_scores = quality_assessor.assess_epochs(epochs)

print(f"\nQuality assessment complete for {len(quality_scores)} epochs")
print(f"First epoch quality: {quality_scores[0]['overall_quality']:.2f}%")


Quality assessment complete for 41 epochs
First epoch quality: 81.91%


In [7]:
# Extract classical features
classical_extractor = ClassicalFeatureExtractor(sampling_rate=128)
classical_features = classical_extractor.extract(epochs[0]['data'])

print(f"\nClassical features extracted")
print(f"Shape: {classical_features.shape}")


Classical features extracted
Shape: (1, 231)


In [8]:
# Extract FMM features
fmm_extractor = FMMFeatureExtractor(n_components=10, sampling_rate=128)
fmm_features = fmm_extractor.extract(epochs[0]['data'])

print(f"\nFMM features extracted")
print(f"Number of components: {fmm_features.shape[0]}")


FMM features extracted
Number of components: 10


## Summary Output (as shown in README)

In [9]:
print(f"Quality Score: {quality_scores[0]['overall_quality']:.2f}%")
print(f"Classical Features Shape: {classical_features.shape}")
print(f"FMM Components: {fmm_features.shape[0]}")

Quality Score: 81.91%
Classical Features Shape: (1, 231)
FMM Components: 10


## Test: Load All Recordings

In [10]:
# Test loading all recordings (use with caution for large datasets)
eeg_data_all, metadata_all = loader.load_patient("0284")
print(f"\nLoaded all recordings: {eeg_data_all.shape}")
print(f"Number of recordings loaded: {metadata_all['num_recordings']}")
print(f"Recording names: {metadata_all['loaded_recordings']}")


Loaded all recordings: (3157000, 20)
Number of recordings loaded: 2
Recording names: ['0284_001_004_ECG', '0284_001_004_EEG']
